In [1]:
  
simulation<-function(){
    d1 = data.frame(cbind("group" = c(rep("ctrl",total_size_ctrl),rep("pth",total_size_pth)), "pre" =base_frequency_fun(total_size_pth+total_size_ctrl)))
    names(d1)<-c("group","pre")
    d1$pre<-as.numeric(d1$pre)
    d1['natural_improver']<-0
    d1[sample(seq(1,dim(d1)[1]),dim(d1)[1]*natural_improvers_pct),'natural_improver']<-1
    d1['intervention']<-0
    d1[sample(row.names(subset(d1, group == "pth")),treated_pct *total_size_pth),'intervention']<-1
    
    d1["post"]<-apply(d1, 1, natural_variability_fun, "pre")
    
    if(sum(d1$intervention) >0){
      d1[d1$intervention ==1,"post"]<-apply(d1[d1$intervention ==1,],1,intervention_improvement_fun, "post")
    }else{
      print('no intervention')
    }
    if(sum(d1$natural_improver) >0){
      d1[d1$natural_improver == 1,"post"]<-apply(d1[d1$natural_improver == 1 ,], 1, natural_improvement_fun, "post")
    }
    if(sum(1- d1$natural_improver) >0){
      d1[d1$natural_improver == 0,"post"]<-apply(d1[d1$natural_improver == 0 ,], 1, natural_variability_fun, "post")
    }
    d1['change']<-d1$pre - d1$post
    d1['change_pct']<-d1$change/d1$pre
    c(as.numeric(unlist(aggregate(change ~ group, d1, mean))[3:4]),
      t.test( subset(d1, group == "pth")$change, subset(d1, group == "ctrl")$change, alternative = "greater")$p.value,
      model_parameters)
}



base_frequency_fun<-function(sz){
  vls<-do.call(rbind,lapply(seq(1,sz,1), FUN = function(normally_distributed_value = NA){
  while (is.na(normally_distributed_value) || normally_distributed_value < pre_min || normally_distributed_value > pre_max) {
    normally_distributed_value <- rnorm(1, mean = pre_mean, sd = pre_sd)
  }
    round(normally_distributed_value,0)
    }))
  vls
}


natural_improvement_fun<-function(x,var){
  #gives a % improvement
  xx = as.numeric(x[[var]])
  max(0,round(xx - xx*runif(1,natural_improvement_min_pct, natural_improvement_max_pct),0))
}



natural_variability_fun<-function(x, var){
  #gives a number 
  xx = as.numeric(x[[var]])
  min(max(0,round(xx + rnorm(1,natural_variability_mean, natural_variability_sd)),0),28)
}

#should intervention make natural decrease even more? 
intervention_improvement_fun<-function(x,var){
  #gives a % improvement
  xx = as.numeric(x[[var]])
  max(0,round(xx - xx*rnorm(1,treatment_improvement_mean, treatment_improvement_sd),0))
}

In [6]:
total_size_ctrl = total_size_pth = 100
#baseline headache frequency parameters
pre_mean = 20
pre_sd = 1.5
pre_min = 15
pre_max = 28

#post treatment headache frequency limits
post_min = 0
post_max = 28

#natural variability. probably this should have a slightly negative mean
natural_variability_mean = 0
natural_variability_sd =0.5

#parameters describing those who naturally resolve
natural_improvement_min_pct = 0.50 
natural_improvement_max_pct = 1
natural_improvers_pct =0.25 # .25

#treatment parameters
treated_pct = 1
treatment_improvement_mean = .2
treatment_improvement_sd = .15

n_replications<-1000
#should it be like a zero inflated? chance of response and then response? 
model_parameters<-c(total_size_ctrl,  pre_mean, pre_sd,pre_min,pre_max, post_min,post_max, natural_variability_mean, natural_variability_sd,
  natural_improvement_min_pct, natural_improvement_max_pct, natural_improvers_pct, treated_pct, treatment_improvement_mean,
  treatment_improvement_sd,n_replications)

res<-as.data.frame(t(replicate(n_replications,simulation())))
names(res)<-c("change_ctrl","change_intervention","p.value", "total_size_ctrl",  "pre_mean", "pre_sd","pre_min","pre_max", "post_min",
              "post_max", "natural_variability_mean", "natural_variability_sd",
              "natural_improvement_min_pct", "natural_improvement_max_pct", "natural_improvers_pct", "treated_pct", "treatment_improvement_mean",
              "treatment_improvement_sd","n_replications" )
res['treatment_effect']<-res$change_intervention - res$change_ctrl

cat("\nmean treatment effect: ", mean(res$treatment_effect))
cat("\nlower limit of treatment effect", mean(res$treatment_effect)  - 2*sd(res$treatment_effect))
cat("\nhow many trials give positive results? ")
table(res$p.value<=0.05)
# 


mean treatment effect:  0.94242lower limit of treatment effect 0.2961397
how many trials give positive results?/n 


FALSE  TRUE 
  102   898 

In [ ]:
#you can ask it to save results with write.csv(res, filepath) where filepath is where and the file name you want to save to 